In [ ]:
import tensorflow as tf

In [ ]:
import numpy as np

In [ ]:
print(tf.__version__)

In [ ]:
tf.test.is_gpu_available()

# 2.2 数据操作

在深度学习中，我们通常会频繁地对数据进行操作。

作为动手学深度学习的基础，本节将介绍如何对内存中的数据进行操作。

在 TensorFlow 中，tensor 是一个类，也是存储和变换数据的主要工具。

如果你之前用过 NumPy，你会发现 tensor 和 NumPy 的多维数组非常类似。然而，tensor 提供 GPU 计算和自动求梯度等更多功能，这些使 tensor 更加适合深度学习。

# 2.2.1 创建 tensor

我们先介绍 `tensor` 的最基本功能，我们用 `arange` 函数创建一个行向量。

In [ ]:
x = tf.constant(range(12))
x

这时返回了一个 `tensor` 实例，其中包含了从 0 开始的 12 个连续整数。

我们可以通过 shape 属性来获取 tensor 实例的形状。

In [ ]:
print(x.shape)

我们也能够通过 len 得到 tensor 实例中元素（element）的总数。

In [ ]:
len(x)

下面使用reshape函数把行向量x的形状改为(3, 4)，也就是一个3行4列的矩阵，并记作X。除了形状改变之外，X中的元素保持不变。



In [ ]:
x = tf.reshape(x,(3,4))

In [ ]:
x

注意X属性中的形状发生了变化。上面x.reshape((3, 4))也可写成x.reshape((-1, 4))或x.reshape((3, -1))。由于x的元素个数是已知的，这里的-1是能够通过元素个数和其他维度的大小推断出来的。

接下来，我们创建一个各元素为0，形状为(2, 3, 4)的张量。实际上，之前创建的向量和矩阵都是特殊的张量。

In [ ]:
X = tf.reshape(x,(-1,4))
X

In [ ]:
tf.zeros((2,3,4))

类似地，我们可以创建各元素为1的张量。



In [ ]:
tf.ones((2,3,4))

In [ ]:
tf.ones((3,4))

我们也可以通过Python的列表（list）指定需要创建的tensor中每个元素的值。

In [ ]:
Y = tf.constant([[1,2,3,4],[6,7,8,9],[10,11,12,13]])
Y

有些情况下，我们需要随机生成tensor中每个元素的值。下面我们创建一个形状为(3, 4)的tensor。它的每个元素都随机采样于均值为0、标准差为1的正态分布。



In [ ]:
tf.random.normal(shape=[3,4], mean=0, stddev=1)

# 2.2.2 运算

tensor支持大量的运算符（operator）。例如，我们可以对之前创建的两个形状为(3, 4)的tensor做按元素加法。所得结果形状不变。

In [ ]:
X + Y

按元素乘法：



In [ ]:
X * Y

按元素除法：


In [ ]:
X / Y

#### 按元素做指数运算：




In [ ]:
Y = tf.cast(Y,tf.float32)
Y

In [ ]:
tf.exp(Y)

除了按元素计算外，我们还可以使用 matmul 函数做矩阵乘法。下面将 X 与 Y 的转置做矩阵乘法。由于X是3行4列的矩阵，Y转置为4行3列的矩阵，因此两个矩阵相乘得到3行3列的矩阵。

In [ ]:
X = tf.cast(X, tf.float32)

In [ ]:
Y = tf.cast(Y, tf.float32)

In [ ]:
tf.matmul(X, tf.transpose(Y))

In [ ]:
tf.matmul(tf.transpose(X), X)

我们也可以将多个 tensor 连结（concatenate）。

下面分别在行上（维度0，即形状中的最左边元素）和列上（维度1，即形状中左起第二个元素）连结两个矩阵。

可以看到，输出的第一个 tensor 在维度0的长度（ 6 ）为两个输入矩阵在维度0的长度之和（ 3+3 ），而输出的第二个 tensor 在维度1的长度（ 8 ）为两个输入矩阵在维度1的长度之和（ 4+4 ）。

In [ ]:
tf.concat([X,Y], axis=0)

In [ ]:
tf.concat([X,Y],axis=1)

使用条件判断式可以得到元素为 0 或 1 的新的tensor。以 X == Y 为例，如果 X 和 Y 在相同位置的条件判断为真（值相等），那么新的tensor在相同位置的值为1；反之为0。

In [ ]:
tf.equal(X,Y)

对 tensor 中的所有元素求和得到只有一个元素的 tensor 。

In [ ]:
tf.reduce_sum(X)

In [ ]:
X

## 求范数

矩阵范数是矩阵 $AA^T$ 最大特征值绝对值的开方

In [ ]:
tf.norm(X)

In [ ]:
tf.linalg.eigh(tf.matmul(X, tf.transpose(X)))

In [ ]:
np.sqrt(5.02176636e+02)

# 广播机制

前面我们看到如何对两个形状相同的 tensor 做按元素运算。

当对两个形状不同的 tensor 按元素运算时，可能会触发广播（broadcasting）机制：先适当复制元素使这两个 tensor 形状相同后再按元素运算。

定义两个 tensor：

In [ ]:
A = tf.reshape(tf.constant(range(3)),(3,1))
B = tf.reshape(tf.constant(range(2)),(1,2))

In [ ]:
A,B

由于 A 和 B 分别是 3 行 1 列和 1 行 2 列的矩阵，如果要计算 A + B，那么 A 中第一列的3个元素被广播（复制）到了第二列，而 B 中第一行的2个元素被广播（复制）到了第二行和第三行。如此，就可以对2个3行2列的矩阵按元素相加。

In [ ]:
A+B

# 2.2.4 索引

在 tensor 中，索引（index）代表了元素的位置。tensor 的索引从 0 开始逐一递增。例如，一个 3 行 2 列的矩阵的行索引分别为 0、1 和 2 ，列索引分别为 0 和 1 。

在下面的例子中，我们指定了 tensor 的行索引截取范围 [1:3] 。依据左闭右开指定范围的惯例，它截取了矩阵 X 中行索引为 1 和 2 的两行。

In [ ]:
X[1:3]

我们可以指定 tensor 中需要访问的单个元素的位置，如矩阵中行和列的索引，并为该元素重新赋值。

In [ ]:
X = tf.Variable(X)

In [ ]:
X[1,2].assign(9)

当然，我们也可以截取一部分元素，并为它们重新赋值。在下面的例子中，我们为行索引为 1 的每一列元素重新赋值。

In [ ]:
X[1,:].assign(tf.ones(X[1,:].shape,dtype=tf.float32)*12)

# 2.2.5 运算的内存开销

在前面的例子里我们对每个操作新开内存来存储运算结果。举个例子，即使像 Y = X + Y 这样的运算，我们也会新开内存，然后将 Y 指向新内存。为了演示这一点，我们可以使用 Python 自带的 id 函数：如果两个实例的 ID 一致，那么它们所对应的内存地址相同；反之则不同。

In [ ]:
before = id(Y)

In [ ]:
before

In [ ]:
Y = Y + X

In [ ]:
id(Y) == before

如果想指定结果到特定内存，我们可以使用前面介绍的索引来进行替换操作。在下面的例子中，我们先通过 zeros_like 创建和 Y 形状相同且元素为 0 的tensor，记为 Z 。

接下来，我们把 X + Y 的结果通过 [:] 写进 Z 对应的内存中。

In [ ]:
Z = tf.Variable(tf.zeros_like(Y))

In [ ]:
Z

In [ ]:
before = id(Z)

In [ ]:
Z[:].assign(X + Y)

In [ ]:
id(Z) == before

实际上，上例中我们还是为 `X + Y` 开了临时内存来存储计算结果，再复制到 `Z` 对应的内存。

如果想避免这个临时内存开销，我们可以使用assign_{运算符全名}函数。

In [ ]:
Z = tf.add(X, Y)
id(Z) == before

In [ ]:
before = id(X)

In [ ]:
X.assign_add(Y)

In [ ]:
id(X) == before

如果 `X` 的值在之后的程序中不会复用，我们也可以用 `X[:] = X + Y` 或者 `X += Y` 来减少运算的内存开销。

# 2.2.6 tensor 和 NumPy 相互变换

我们可以通过 array 函数和 asnumpy 函数令数据在 NDArray 和 NumPy 格式之间相互变换。下面将 NumPy 实例变换成 tensor 实例。

In [ ]:
P = np.ones((2,3))

In [ ]:
D = tf.constant(P)

In [ ]:
D

再将 NDArray 实例变换成 NumPy 实例。

In [ ]:
np.array(D)